In [1]:
%matplotlib inline
from hurricane_gis import load_best_track

- putting everything together to build GIS App for Michael
- fetching data using `pyoos`
- interactive maps (`folium`)
- HTML plots (`bokeh`)

In [2]:
from IPython.display import IFrame

IFrame('https://ioos.github.io/notebooks_demos/code_gallery', width='100%', height=450)

![XKCD data pipeline](https://imgs.xkcd.com/comics/data_pipeline.png)

[https://xkcd.com/2054](https://xkcd.com/2054/)

In [3]:
radii, pts = load_best_track(code='al14', year='2018')

start = radii.index[0]
end = radii.index[-1]
bbox = tuple(radii['geometry'].total_bounds)

In [4]:
strbbox = ', '.join(format(v, '.2f') for v in bbox)
print(f'bbox: {strbbox}\nstart: {start}\n  end: {end}')

bbox: -89.23, 15.71, -69.99, 38.80
start: 2018-10-07 12:00:00
  end: 2018-10-12 06:00:00


In [5]:
import shapely


coords = zip(pts['LON'], pts['LAT'])
track = shapely.geometry.LineString(coords)

In [6]:
import cf_units
from ioos_tools.ioos import collector2table
import pandas as pd
from pyoos.collectors.coops.coops_sos import CoopsSos
from retrying import retry


# We need to retry in case of failure b/c the server cannot handle
# the high traffic during events like hurricanes.
@retry(stop_max_attempt_number=5, wait_fixed=3000)
def get_coops(start, end, sos_name, units, bbox, verbose=False):
    collector = CoopsSos()
    collector.set_bbox(bbox)
    collector.end_time = end
    collector.start_time = start
    collector.variables = [sos_name]
    ofrs = collector.server.offerings
    title = collector.server.identification.title
    config = dict(
        units=units,
        sos_name=sos_name,
    )

    data = collector2table(
        collector=collector,
        config=config,
        col=f'{sos_name} ({units.format(cf_units.UT_ISO_8859_1)})'
    )

    # Clean the table.
    table = dict(
        station_name=[s._metadata.get('station_name') for s in data],
        station_code=[s._metadata.get('station_code') for s in data],
        sensor=[s._metadata.get('sensor') for s in data],
        lon=[s._metadata.get('lon') for s in data],
        lat=[s._metadata.get('lat') for s in data],
        depth=[s._metadata.get('depth', 'NA') for s in data],
    )

    table = pd.DataFrame(table).set_index('station_name')
    if verbose:
        print('Collector offerings')
        print(f'{title}: {len(ofrs)} offerings')
    return data, table

In [7]:
ssh, ssh_table = get_coops(
    start=start,
    end=end,
    sos_name='water_surface_height_above_reference_datum',
    units=cf_units.Unit('meters'),
    bbox=bbox,
)

ssh_table.tail()

,station_code,sensor,lon,lat,depth
station_name,,,,,
"Chickasaw Creek, AL",8737138,urn:ioos:sensor:NOAA.NOS.CO-OPS:8737138:Y1,-88.0736,30.7819,None
"West Fowl River Bridge, AL",8738043,urn:ioos:sensor:NOAA.NOS.CO-OPS:8738043:Y1,-88.1586,30.3766,None
"Bayou La Batre Bridge, AL",8739803,urn:ioos:sensor:NOAA.NOS.CO-OPS:8739803:Y1,-88.2478,30.4062,None
"Grand Bay NERR, Mississippi Sound, MS",8740166,urn:ioos:sensor:NOAA.NOS.CO-OPS:8740166:A1,-88.4029,30.4132,None
"Pascagoula NOAA Lab, MS",8741533,urn:ioos:sensor:NOAA.NOS.CO-OPS:8741533:A1,-88.5631,30.3678,None


In [8]:
wind_speed, wind_speed_table = get_coops(
    start=start,
    end=end,
    sos_name='wind_speed',
    units=cf_units.Unit('m/s'),
    bbox=bbox,
)

wind_speed_table.tail()

,station_code,sensor,lon,lat,depth
station_name,,,,,
"Pensacola, FL",8729840,urn:ioos:sensor:NOAA.NOS.CO-OPS:8729840:C1,-87.2112,30.4044,None
"Fort Morgan, AL",8734673,urn:ioos:sensor:NOAA.NOS.CO-OPS:8734673:C1,-88.0250,30.2283,None
"Dauphin Island, AL",8735180,urn:ioos:sensor:NOAA.NOS.CO-OPS:8735180:C1,-88.0750,30.2500,None
"Coast Guard Sector Mobile, AL",8736897,urn:ioos:sensor:NOAA.NOS.CO-OPS:8736897:C1,-88.0583,30.6483,None
"Petit Bois Island, Port of Pascagoula, MS",8741003,urn:ioos:sensor:NOAA.NOS.CO-OPS:8741003:C1,-88.5000,30.2133,None


In [9]:
common = set(ssh_table['station_code']).intersection(wind_speed_table['station_code'])

In [10]:
ssh_obs, win_obs = [], []

for station in common:
    ssh_obs.extend([obs for obs in ssh if obs._metadata['station_code'] == station])
    win_obs.extend([obs for obs in wind_speed if obs._metadata['station_code'] == station])

In [11]:
index = pd.date_range(
    start=start.replace(tzinfo=None),
    end=end.replace(tzinfo=None),
    freq='15min'
)

ssh_observations = []
for series in ssh_obs:
    _metadata = series._metadata
    obs = series.reindex(index=index, limit=1, method='nearest')
    obs._metadata = _metadata
    obs.name = _metadata['station_name']
    ssh_observations.append(obs)

winds_observations = []
for series in win_obs:
    _metadata = series._metadata
    obs = series.reindex(index=index, limit=1, method='nearest')
    obs._metadata = _metadata
    obs.name = _metadata['station_name']
    winds_observations.append(obs)

In [12]:
from bokeh.resources import CDN
from bokeh.plotting import figure
from bokeh.embed import file_html
from bokeh.models import Range1d, LinearAxis, HoverTool

from folium import IFrame

# Plot defaults.
tools = "pan,box_zoom,reset"
width, height = 750, 250

In [13]:
def make_plot(ssh, wind):
    p = figure(
        toolbar_location='above', x_axis_type='datetime',
        width=width, height=height, tools=tools, title=ssh.name)

    p.yaxis.axis_label = 'wind speed (m/s)'
    l0 = p.line(
        x=wind.index, y=wind.values, line_width=5,
        line_cap='round', line_join='round',
        legend='wind speed (m/s)', color='#9900cc', alpha=0.5)

    p.extra_y_ranges = {}
    p.extra_y_ranges['y2'] = Range1d(start=-1, end=3.5)

    p.add_layout(LinearAxis(y_range_name='y2',
                            axis_label='ssh (m)'), 'right')

    l1 = p.line(
        x=ssh.index, y=ssh.values, line_width=5, line_cap='round',
        line_join='round', legend='ssh (m)', color='#0000ff',
        alpha=0.5, y_range_name='y2')

    p.legend.location = 'top_left'
    p.add_tools(
        HoverTool(tooltips=[('wind speed (m/s)', '@y'),],
                  renderers=[l0]),
        HoverTool(tooltips=[('ssh (m)', '@y'),],
                  renderers=[l1]))
    return p

In [14]:
def make_marker(p, location, fname):
    html = file_html(p, CDN, fname)
    iframe = IFrame(html, width=width+45, height=height+80)

    popup = folium.Popup(iframe, max_width=2650)
    icon = folium.Icon(color='green', icon='stats')
    marker = folium.Marker(location=location,
                           popup=popup,
                           icon=icon)
    return marker

In [15]:
import folium
from folium.plugins import Fullscreen, MarkerCluster
from ioos_tools.ioos import get_coordinates


lon = track.centroid.x
lat = track.centroid.y

m = folium.Map(location=[lat, lon], tiles='OpenStreetMap', zoom_start=4)

Fullscreen(position='topright', force_separate_button=True).add_to(m)

marker_cluster0 = MarkerCluster(name='Observations')
marker_cluster0.add_to(m);

In [16]:
url = 'http://oos.soest.hawaii.edu/thredds/wms/hioos/satellite/dhw_5km'
w0 = folium.WmsTileLayer(
    url,
    name='Sea Surface Temperature',
    fmt='image/png',
    layers='CRW_SST',
    attr='PacIOOS TDS',
    overlay=True,
    transparent=True)

url = 'http://hfrnet.ucsd.edu/thredds/wms/HFRNet/USEGC/6km/hourly/RTV'
w1 = folium.WmsTileLayer(
    url,
    name='HF Radar',
    fmt='image/png',
    layers='surface_sea_water_velocity',
    attr='HFRNet',
    overlay=True,
    transparent=True)

w0.add_to(m)
w1.add_to(m);

In [17]:
colors = {
    'EX': 'yellow',
    'TD': 'yellow',
    'TS': 'orange',
    'HU': 'red',
}

In [18]:
def style_function(feature):
    return {
        'fillOpacity': 0,
        'color': 'black',
        'stroke': 1,
        'weight': 0.5,
        'opacity': 0.2,
    }


for date, row in pts.iterrows():
    storm_type = row['STORMTYPE']
    location = row['LAT'], row['LON']
    popup = f'{date}<br>{storm_type}'
    folium.CircleMarker(
        location=location,
        radius=10,
        fill=True,
        color=colors[storm_type],
        popup=popup,
    ).add_to(m)

In [19]:
for ssh, wind in zip(ssh_observations, winds_observations):
    fname = ssh._metadata['station_code']
    location = ssh._metadata['lat'], ssh._metadata['lon']
    p = make_plot(ssh, wind)
    marker = make_marker(p, location=location, fname=fname)
    marker.add_to(marker_cluster0)

folium.LayerControl().add_to(m)

p = folium.PolyLine(get_coordinates(bbox),
                    color='#009933',
                    weight=1,
                    opacity=0.2)

p.add_to(m);

In [20]:
def embed_map(m):
    from IPython.display import HTML

    m.save('index.html')
    with open('index.html') as f:
        html = f.read()

    srcdoc = html.replace('"', '&quot;')
    iframe = f'<iframe srcdoc="{srcdoc}" style="width: 100%; height: 750px; border: none"></iframe>'
    return HTML(iframe)

In [21]:
embed_map(m)